## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [13]:
f.predictions('nvda')

NVDA 5m Interval Timestamp: 2024-11-13 03:19:17 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.372329,0.355797,0.346815,0.176238
prob_up,0.23624,0.161898,0.221346,0.25
prob_static,0.526758,0.580708,0.471952,0.375
prob_down,0.237002,0.257395,0.306702,0.375
precision,0.551664,0.539855,0.533439,0.411598
recall,0.59434,0.562264,0.632075,0.549057
f1,0.572207,0.550832,0.578584,0.470493
support,"[530.0, 530.0, 529.0]","[530.0, 530.0, 529.0]","[530.0, 530.0, 529.0]","[530.0, 530.0, 529.0]"


NVDA 15m Interval Timestamp: 2024-11-13 03:19:24 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,up
kelly_1:2.5,0.285106,0.324138,0.292929,0.124022
prob_up,0.325987,0.267384,0.340621,0.5
prob_static,0.491442,0.514777,0.387286,0.25
prob_down,0.182571,0.217838,0.272093,0.25
precision,0.489362,0.517241,0.494949,0.374302
recall,0.525714,0.514286,0.56,0.378531
f1,0.506887,0.515759,0.525469,0.376404
support,"[177.0, 175.0, 174.0]","[177.0, 175.0, 174.0]","[177.0, 175.0, 174.0]","[177.0, 175.0, 174.0]"


NVDA 1h Interval Timestamp: 2024-11-13 03:19:27 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,down,static,static
kelly_1:2.5,0.061176,0.1179,0.091963,0.069799
prob_up,0.292722,0.307321,0.285319,0.375
prob_static,0.36633,0.324085,0.373201,0.375
prob_down,0.340948,0.368595,0.34148,0.25
precision,0.329412,0.369928,0.351402,0.33557
recall,0.388889,0.359629,0.435185,0.462963
f1,0.356688,0.364706,0.388831,0.389105
support,"[430.0, 432.0, 431.0]","[430.0, 432.0, 431.0]","[430.0, 432.0, 431.0]","[430.0, 432.0, 431.0]"


NVDA 1wk Interval Timestamp: 2024-11-13 03:19:40 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,down
kelly_1:2.5,0.082759,0.01087,0.093182,0.066667
prob_up,0.319165,0.287682,0.30957,0.125
prob_static,0.517225,0.483976,0.398219,0.25
prob_down,0.16361,0.228342,0.292211,0.625
precision,0.344828,0.293478,0.352273,0.333333
recall,0.340909,0.306818,0.352273,0.238636
f1,0.342857,0.3,0.352273,0.278146
support,"[88.0, 88.0, 88.0]","[88.0, 88.0, 88.0]","[88.0, 88.0, 88.0]","[88.0, 88.0, 88.0]"


NVDA 1mo Interval Timestamp: 2024-11-13 03:19:42 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,down,static,static
kelly_1:2.5,0.2125,-0.166667,0.066667,0.0375
prob_up,0.435812,0.223594,0.258724,0.125
prob_static,0.388211,0.361718,0.418201,0.625
prob_down,0.175977,0.414687,0.323075,0.25
precision,0.4375,0.166667,0.333333,0.3125
recall,0.388889,0.111111,0.4,0.5
f1,0.411765,0.133333,0.363636,0.384615
support,"[18.0, 20.0, 18.0]","[18.0, 20.0, 18.0]","[18.0, 20.0, 18.0]","[18.0, 20.0, 18.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')